In [21]:
from typing import Optional
import logging
from dotenv import load_dotenv
import json
import re

from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool
from langchain_core.output_parsers import PydanticToolsParser, PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough

In [2]:
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

load_dotenv()

True

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Structured Output

## Tool Calling

In [4]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse.', rating=7)

## Nested Schema (No Parallelism)

In [5]:
class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")

In [6]:
class Response(BaseModel):
    output: Joke | ConversationalResponse

In [7]:
structured_llm = llm.with_structured_output(Response)

In [8]:
structured_llm.invoke("Tell me a joke about cats")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(output=Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse.', rating=8))

In [9]:
structured_llm.invoke("How are you today?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(output=ConversationalResponse(response="I'm just a computer program, so I don't have feelings, but I'm here to help you with anything you need. How can I assist you today?"))

## Streaming

In [10]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

In [11]:
for chunk in structured_llm.stream("Tell me a joke about cats"):
    print(chunk)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why was'}
{'setup': 'Why was the'}
{'setup': 'Why was the cat'}
{'setup': 'Why was the cat sitting'}
{'setup': 'Why was the cat sitting on'}
{'setup': 'Why was the cat sitting on the'}
{'setup': 'Why was the cat sitting on the computer'}
{'setup': 'Why was the cat sitting on the computer?'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': ''}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'To'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'To keep'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'To keep an'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'To keep an eye'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'To keep an eye on'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'To keep an eye on the'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'To keep an eye on the mouse

## Few-Shot Learning

In [12]:
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'setup': 'Wooden shoe',
 'punchline': 'Wooden shoe like to know where all the woodpeckers have gone!',
 'rating': 8}

In [13]:
examples = [
    HumanMessage("Tell me a joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Why don't planes ever get tired?",
                    "punchline": "Because they have rest wings!",
                    "rating": 2,
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
    HumanMessage("Tell me another joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Cargo",
                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                    "rating": 10,
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Now about caterpillars", name="example_user"),
    AIMessage(
        "",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Caterpillar",
                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                    "rating": 5,
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?")."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'setup': 'Crocodile',
 'punchline': "Crocodile that's who! Don't mess with me, I snap back!",
 'rating': 7}

## JSON Mode (Not Recommended)

In [14]:
# If using JSON mode you'll have to still specify the desired schema in the model prompt. The schema you pass to with_structured_output will only be used for parsing the model outputs, it will not be passed to the model the way it is with tool calling.
structured_llm = llm.with_structured_output(Joke, method="json_mode")

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Joke(setup='Why was the cat sitting on the computer?', punchline='It wanted to keep an eye on the mouse!', rating=None)

## Raw Outputs

In [15]:
structured_llm = llm.with_structured_output(Joke, include_raw=True)

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WjbGYBOO8rd5HDq8yK4BhNr8', 'function': {'arguments': '{"setup":"Why was the cat sitting on the computer?","punchline":"It wanted to keep an eye on the mouse!"}', 'name': 'Joke'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 114, 'total_tokens': 142}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0072665b-395c-4e64-be7c-5bc2801f985d-0', tool_calls=[{'name': 'Joke', 'args': {'setup': 'Why was the cat sitting on the computer?', 'punchline': 'It wanted to keep an eye on the mouse!'}, 'id': 'call_WjbGYBOO8rd5HDq8yK4BhNr8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 114, 'output_tokens': 28, 'total_tokens': 142}),
 'parsed': Joke(setup='Why was the cat sitting on the computer?', punchline='It wanted to keep an eye on the mouse!', rating=None),
 'parsing_error': None}

## Schema in Prompt (Not Recommended)

In [16]:
class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: list[Person]


# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

In [17]:
query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.invoke(query).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Identifying information about all people in a text.", "properties": {"people": {"title": "People", "type": "array", "items": {"$ref": "#/definitions/Person"}}}, "required": ["people"], "definitions": {"Person": {"title": "Person", "description": "Information about a person.", "type": "object", "properties": {"name": {"title": "Name", "description": "The name of the person", "type": "string"}, "height_in_meters": {"title": "Height In Meters", "description": "The heig

In [18]:
chain = prompt | llm | parser

chain.invoke({"query": query})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


People(people=[Person(name='Anna', height_in_meters=1.83)])

## Custom Parsing

In [19]:
class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: list[Person]


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Output your answer as JSON that  "
            "matches the given schema: ```json\n{schema}\n```. "
            "Make sure to wrap the answer in ```json and ``` tags",
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.schema())


# Custom parser
def extract_json(message: AIMessage) -> list[dict]:
    """Extracts JSON content from a string where JSON is embedded between ```json and ``` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    text = message.content
    # Define the regular expression pattern to match JSON blocks
    pattern = r"```json(.*?)```"

    # Find all non-overlapping matches of the pattern in the string
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the list of matched JSON strings, stripping any leading or trailing whitespace
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

In [20]:
query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.format_prompt(query=query).to_string())

System: Answer the user query. Output your answer as JSON that  matches the given schema: ```json
{'title': 'People', 'description': 'Identifying information about all people in a text.', 'type': 'object', 'properties': {'people': {'title': 'People', 'type': 'array', 'items': {'$ref': '#/definitions/Person'}}}, 'required': ['people'], 'definitions': {'Person': {'title': 'Person', 'description': 'Information about a person.', 'type': 'object', 'properties': {'name': {'title': 'Name', 'description': 'The name of the person', 'type': 'string'}, 'height_in_meters': {'title': 'Height In Meters', 'description': 'The height of the person expressed in meters.', 'type': 'number'}}, 'required': ['name', 'height_in_meters']}}}
```. Make sure to wrap the answer in ```json and ``` tags
Human: Anna is 23 years old and she is 6 feet tall


In [21]:
chain = prompt | ChatOpenAI(model="gpt-4o") | extract_json

chain.invoke({"query": query})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[{'title': 'People',
  'description': 'Identifying information about all people in a text.',
  'type': 'object',
  'properties': {'people': {'title': 'People',
    'type': 'array',
    'items': {'$ref': '#/definitions/Person'}}},
  'required': ['people'],
  'definitions': {'Person': {'title': 'Person',
    'description': 'Information about a person.',
    'type': 'object',
    'properties': {'name': {'title': 'Name',
      'description': 'The name of the person',
      'type': 'string'},
     'height_in_meters': {'title': 'Height In Meters',
      'description': 'The height of the person expressed in meters.',
      'type': 'number'}},
    'required': ['name', 'height_in_meters']}},
  'people': [{'name': 'Anna', 'height_in_meters': 1.8288}]}]

In [22]:
# https://python.langchain.com/v0.2/docs/how_to/structured_output/

# Function/Tool Calling

In [4]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [5]:
# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

In [6]:
llm_with_tools = llm.bind_tools(tools)

In [7]:
query = "What is 3 * 12?"

llm_with_tools.invoke(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f1PxtpRgftli2RwvsStMsVfB', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 95, 'total_tokens': 113}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-02e3fc6a-7083-497d-a6c1-bb04bb109484-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_f1PxtpRgftli2RwvsStMsVfB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 18, 'total_tokens': 113})

In [8]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[{'name': 'Multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_HTNCLsCAYrblmsGVLYVPtzfw',
  'type': 'tool_call'},
 {'name': 'Add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_gpAnjFsjoJi2EnGlzYkzEV60',
  'type': 'tool_call'}]

In [9]:
chain = llm_with_tools | PydanticToolsParser(tools=[Multiply, Add])
chain.invoke(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[Multiply(a=3, b=12), Add(a=11, b=49)]

In [10]:
chain.invoke("what is 12 + 50")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[Add(a=12, b=50)]

## Streaming

In [11]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[]
[{'name': 'Multiply', 'args': '', 'id': 'call_U4OQHBNj2sXG9LJgfkygmWKt', 'index': 0}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 0}]
[{'name': None, 'args': ': 3, ', 'id': None, 'index': 0}]
[{'name': None, 'args': '"b": 1', 'id': None, 'index': 0}]
[{'name': None, 'args': '2}', 'id': None, 'index': 0}]
[{'name': 'Add', 'args': '', 'id': 'call_vzgujgEVypRTk6ycOyzzT05t', 'index': 1}]
[{'name': None, 'args': '{"a"', 'id': None, 'index': 1}]
[{'name': None, 'args': ': 11,', 'id': None, 'index': 1}]
[{'name': None, 'args': ' "b": ', 'id': None, 'index': 1}]
[{'name': None, 'args': '49}', 'id': None, 'index': 1}]
[]


In [12]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_call_chunks)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[]
[{'name': 'Multiply', 'args': '', 'id': 'call_9yYoF74VeFBjybqScmXyhzA7', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a"', 'id': 'call_9yYoF74VeFBjybqScmXyhzA7', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, ', 'id': 'call_9yYoF74VeFBjybqScmXyhzA7', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 1', 'id': 'call_9yYoF74VeFBjybqScmXyhzA7', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_9yYoF74VeFBjybqScmXyhzA7', 'index': 0}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_9yYoF74VeFBjybqScmXyhzA7', 'index': 0}, {'name': 'Add', 'args': '', 'id': 'call_7OnQPGvdRRXbyYIMPKob0Jbn', 'index': 1}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_9yYoF74VeFBjybqScmXyhzA7', 'index': 0}, {'name': 'Add', 'args': '{"a"', 'id': 'call_7OnQPGvdRRXbyYIMPKob0Jbn', 'index': 1}]
[{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_9yYoF74VeFBjybqScmXyhzA7', 'index': 0}, {'name': 'Add', 'args': '{"a": 11,', 'id': 'call_

In [13]:
print(type(gathered.tool_call_chunks[0]))
print(gathered.tool_call_chunks[0])

<class 'dict'>
{'name': 'Multiply', 'args': '{"a": 3, "b": 12}', 'id': 'call_9yYoF74VeFBjybqScmXyhzA7', 'index': 0}


In [14]:
first = True
async for chunk in llm_with_tools.astream(query):
    if first:
        gathered = chunk
        first = False
    else:
        gathered = gathered + chunk

    print(gathered.tool_calls)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[]
[{'name': 'Multiply', 'args': {}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}]
[{'name': 'Multiply', 'args': {}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}]
[{'name': 'Multiply', 'args': {'a': 3}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 1}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}, {'name': 'Add', 'args': {}, 'id': 'call_mluQdWP54le5rYboGvZz543n'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}, {'name': 'Add', 'args': {}, 'id': 'call_mluQdWP54le5rYboGvZz543n'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}, {'name': 'Add', 'args': {'a': 11}, 'id': 'call_mluQdWP54le5rYboGvZz543n'}]
[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}, {'name': 'Add', 'args': {

In [15]:
print(type(gathered.tool_calls[0]))
print(gathered.tool_calls[0])

<class 'dict'>
{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_tUJ3Kni8UBW1fdhgVHOaOpNx'}


## Passing tool outputs to model

In [16]:
messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [17]:
messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_65AUsuXSCS9oB4WlGPw5bzp8', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'Multiply'}, 'type': 'function'}, {'id': 'call_CHiyDLA9cOQNwVwUXjrw8Biw', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'Add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 105, 'total_tokens': 155}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d77aaeb1-ba55-4b25-bd67-f504a854a0c1-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_65AUsuXSCS9oB4WlGPw5bzp8', 'type': 'tool_call'}, {'name': 'Add', 'args': {'a': 11, 'b': 49}, 'id': 'call_CHiyDLA9cOQNwVwUXjrw8Biw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 105, 'output_tokens': 50, 'total_tokens': 155}),
 ToolMessage(content='36', tool_call_id='call_6

In [18]:
messages[-2:]

[ToolMessage(content='36', tool_call_id='call_65AUsuXSCS9oB4WlGPw5bzp8'),
 ToolMessage(content='60', tool_call_id='call_CHiyDLA9cOQNwVwUXjrw8Biw')]

In [19]:
llm_with_tools.invoke(messages)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='3 * 12 is 36 and 11 + 49 is 60.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 171, 'total_tokens': 189}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fc2dfd84-24f5-4db9-893c-e25037ab6116-0', usage_metadata={'input_tokens': 171, 'output_tokens': 18, 'total_tokens': 189})

## Few-shot prompting

In [22]:
# Wrong answer
llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"
).tool_calls

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_aXN8LMyObpM31FSVR3VxOVRV',
  'type': 'tool_call'},
 {'name': 'Add',
  'args': {'a': 952, 'b': -20},
  'id': 'call_gS9UVJJsojvPrRlesmdt3WhN',
  'type': 'tool_call'}]

In [23]:
examples = [
    HumanMessage(
        "What's the product of 317253 and 128472 plus four", name="example_user"
    ),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {"name": "Multiply", "args": {"x": 317253, "y": 128472}, "id": "1"}
        ],
    ),
    ToolMessage("16505054784", tool_call_id="1"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{"name": "Add", "args": {"x": 16505054784, "y": 4}, "id": "2"}],
    ),
    ToolMessage("16505054788", tool_call_id="2"),
    AIMessage(
        "The product of 317253 and 128472 plus four is 16505054788",
        name="example_assistant",
    ),
]

system = """You are bad at math but are an expert at using a calculator. 

Use past tool usage as an example of how to correctly use the tools."""
few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        *examples,
        ("human", "{query}"),
    ]
)

chain = {"query": RunnablePassthrough()} | few_shot_prompt | llm_with_tools
chain.invoke("Whats 119 times 8 minus 20").tool_calls

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[{'name': 'Multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_eMcwplZtUDDgoaAKGrMGXM03',
  'type': 'tool_call'}]

In [24]:
chain.invoke("Whats 119 times 8 minus 20")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RjZgpRYfEaCyDgbjKNQ3J675', 'function': {'arguments': '{"a":119,"b":8}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 243, 'total_tokens': 261}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c89e9ef0-797a-43cf-9910-81517e0af370-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 119, 'b': 8}, 'id': 'call_RjZgpRYfEaCyDgbjKNQ3J675', 'type': 'tool_call'}], usage_metadata={'input_tokens': 243, 'output_tokens': 18, 'total_tokens': 261})

In [25]:
# https://python.langchain.com/v0.2/docs/how_to/function_calling/